<a href="https://colab.research.google.com/github/AlenaAntipina/PytorchLearning/blob/main/KungfuMasterDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount your Google Drive in Colab

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
!ln -s /content/gdrive/Мой\ Диск/ /mydrive

Setup Darknet

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15424, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 15424 (delta 0), reused 1 (delta 0), pack-reused 15423
Receiving objects: 100% (15424/15424), 14.03 MiB | 21.44 MiB/s, done.
Resolving deltas: 100% (10365/10365), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [ ]:
# make darknet 
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

Loading pretrained weights

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2022-06-22 12:25:21--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  20.7MB/s    in 12s     

2022-06-22 12:25:34 (19.2 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
!cp /mydrive/CV/yolov4.weights 

cp: cannot stat '/mydrive/CV/yolov4.weights': No such file or directory


Load our custom data

In [ ]:
!cp /mydrive/CV/Dataset.zip ../

cp: cannot stat '/mydrive/CV/Dataset.zip': No such file or directory


In [ ]:
!unzip ../Dataset.zip -d data/

unzip:  cannot find or open ../Dataset.zip, ../Dataset.zip.zip or ../Dataset.zip.ZIP.


Additional files with dataset description

In [ ]:
def add_txt_files_with_names_of_images(split = "train"):
  import os 
  image_files = []
  os.chdir(os.path.join("data", split))
  for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
      image_files.append(f"data/{split}/" + filename)
  os.chdir("..")
  with open(f"{split}.txt", "w") as outfile:
    for image in image_files:
      outfile.write(image)
      outfile.write("\n")
    outfile.close()
  os.chdir("..")

In [ ]:
for split in ["train", "valid"]:
  add_txt_files_with_names_of_images(split)

In [ ]:
!ls data/

9k.tree      goal.txt		       person.jpg	    train.txt
coco9k.map   horses.jpg		       README.dataset.txt   valid
coco.names   imagenet.labels.list      README.roboflow.txt  valid.txt
dog.jpg      imagenet.shortnames.list  scream.jpg	    voc.names
eagle.jpg    labels		       test
giraffe.jpg  openimages.names	       train


Male the directory for backup on GoogleDrive

Setup configs of the model

In [ ]:
# comment if already execute it
!cp cfg/yolov4-custom.cfg /mydrive/CV/yolov4-obj.cfg

Change configuration

width = 416

height = 416

max_batches = 6000 (number of classes x 2000, not less than 6000)  

steps = 4800,5400 (80% and 90% of max_batches)

filters = 24  ((number of classes + 5) x 3, change in each yolo)

classes = 3


IF CUDA has no enough memory (try step by step)
1. subdivisions = 64 (instead of 16)
2. batch = 16 (instead of 64)
3. random = 0 (instead of 1)

Make files with description of data

Make in GoogleDrive 2 files: obj.data and obj.names

obj.names contains 3 names of classes - me, missile, opponent1

Copy files from google drive to colab

In [ ]:
!cp /mydrive/CV/yolov4-obj.cfg cfg/yolov4-obj.cfg

In [ ]:
!cp /mydrive/CV/obj.names ./data

In [ ]:
!cp /mydrive/CV/obj.data ./data

Training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov3.weights -dont_show -map

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn

Check metrics

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/CV/backup/yolov4-obj_final.weights

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv  

# Новый раздел